For this workshop, you need:

* An Azure Machine Learning workspace. 
* The Azure Machine Learning Python SDK v2 installed. 

To install the SDK you can either,

Create a compute instance, which already has installed the latest AzureML Python SDK and is pre-configured for ML workflows.

Use the followings commands to install Azure ML Python SDK v2:

```bash
pip install azure-ai-ml --upgrade
```

## Connect to ML Client

To connect to a workspace, you need to provide a subscription, resource group and workspace name. These details are used in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace.

In the following example, the default Azure authentication is used along with the default workspace configuration or from any `config.json` file you might have copied into the folders structure. If no `config.json` is found, then you need to manually introduce the subscription_id, resource_group and workspace when creating `MLClient`.

```python
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AzureML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AZUREML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
```


In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

# Retrieve details of Azure ML workspace from environment variables of your Compute Instance
subscription_id = re.search("subscriptions/(.*)/resourceGroups", os.environ["MLFLOW_TRACKING_URI"]).group(
    1
)  # Extract Azure Subcription ID from MLFlow Tracking URI
resource_group = os.environ["CI_RESOURCE_GROUP"]
workspace_name = os.environ["CI_WORKSPACE"]

# Connect to Azure ML workspace
ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

# Online Endpoint

Online endpoints are endpoints that are used for online (real-time) inferencing. They receive data from clients and can send responses back in real time.

An **endpoint** is an HTTPS endpoint that clients can call to receive the inferencing (scoring) output of a trained model. It provides:
* Authentication using "key & token" based auth
* SSL termination
* A stable scoring URI (endpoint-name.region.inference.ml.azure.com)

A **deployment** is a set of resources required for hosting the model that does the actual inferencing.
A single endpoint can contain multiple deployments.

Features of the managed online endpoint:

* **Test and deploy locally** for faster debugging
* Traffic to one deployment can also be **mirrored** (copied) to another deployment.
* **Application Insights integration**
* Security
* Authentication: Key and Azure ML Tokens
* Automatic Autoscaling
* Visual Studio Code debugging

**blue-green deployment**: An approach where a new version of a web service is introduced to production by deploying it to a small subset of users/requests before deploying it fully.

<center>
<img src="../../../imgs/concept_online_endpoint.png" width = "500px" alt="Online Endpoint Concept cli vs sdk">
</center>

## 1. Create Online Endpoint

We can create an **online endpoint** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../../imgs/create_online_endpoint.png" width = "700px" alt="Create Online Endpoint cli vs sdk">
</center>

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

# create an online endpoint
online_endpoint = ManagedOnlineEndpoint(
    name=### TO DO ###, 
    description=### TO DO ###,
)
ml_client.online_endpoints.begin_create_or_update(
    online_endpoint,   
)

## 2. Create Online Deployment

To create a deployment to online endpoint, you need to specify the following elements:

* Model files (or specify a registered model in your workspace)
* Scoring script - code needed to do scoring/inferencing
* Environment - a Docker image with Conda dependencies, or a dockerfile
* Compute instance & scale settings

Note that if you're deploying **MLFlow models**, there's no need to provide **a scoring script** and execution **environment**, as both are autogenerated.

We can create an **online deployment** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../../imgs/create_online_deployment.png" width = "700px" alt="Create Online Deployment cli vs sdk">
</center>

In [ ]:
# create online deployment
from azure.ai.ml.entities import ManagedOnlineDeployment, Model, Environment
from azure.ai.ml.entities import (
    Environment,
    CodeConfiguration,
)

env = Environment(
    conda_file="../../../data-science/environment/deploy-online-conda.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

blue_deployment = ManagedOnlineDeployment(
    name=### TO DO ###,
    endpoint_name=### TO DO ###,
    model=### TO DO ###,
    environment=### TO DO ###,
    code_configuration=CodeConfiguration(
        code="../../../data-science/src/score", scoring_script=### TO DO ###
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment
)


## 3. Allocate Traffic

In [ ]:
# allocate traffic
# new deployment takes 100 traffic
online_endpoint.traffic = {### TO DO ###: ### TO DO ###}
ml_client.begin_create_or_update(online_endpoint)

## 4. Invoke and Test Endpoint

We can invoke the **online deployment** with cli v2 or sdk v2 using the following syntax:

<center>
<img src="../../../imgs/invoke_online_endpoint.png" width = "700px" alt="Invoke online endpoint cli vs sdk">
</center>

In [ ]:
# invoke and test endpoint
ml_client.online_endpoints.invoke(
    endpoint_name=### TO DO ###,
    request_file="../../../data/taxi-request.json",
)